In [2]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

import datetime as dt

#https://colab.research.google.com/github/uwdata/visualization-curriculum/blob/master/altair_introduction.ipynb#scrollTo=Zad2hV5G72j2

In [3]:
name = 'TSLA'
# date = dt.date.today()
date = dt.date(2022,5,27)
df = pd.read_csv('./data/{}/{}{}{}'.format(name, name, date,'close'))

chart = alt.Chart(df)



In [4]:
# Common Functions

import numpy as np
from scipy import stats
import sys
def optionprice(type,S,K,tau,sig,dft=0):
    '''
    type : option type: 'CALL' or 'PUT'
    S: stock price: float/int
    K: strike price: float/int
    tau: days left to expiration: float/int
    sig: volatility: float/int, without %
    dft: drift rate in years (due to bias or inflation): float, default=0, best match for TSLA around 0.02
    '''
    
    tau = max(tau,0)/ 365 # turn in to years, avoid negative
    sig = sig/100 # add %
    
    if type=='CALL':
        if S == 0:  # this is to avoid log(0) issues
            return 0.0
        elif tau == 0 or sig == 0:  # this is to avoid 0/0 issues
            return max(S - K, 0)
        else:
            d = (np.log(S / K) + dft * tau) / (sig * np.sqrt(tau))
            
            d1 = d + sig * np.sqrt(tau) / 2

            d2 = d - sig * np.sqrt(tau) / 2

            price =  np.exp(dft * tau) * S * stats.norm.cdf(d1, 0.0, 1.0) - K * stats.norm.cdf(d2, 0.0, 1.0)
        return price
        
    elif type == 'PUT':
        if S == 0:  # this is to avoid log(0) issues
            return 0.0
        elif tau == 0 or sig == 0:  # this is to avoid 0/0 issues
            return max(K - S, 0)
        else:
            d = (np.log(S / K) + dft * tau) / (sig * np.sqrt(tau))

            d1 = -d + sig * np.sqrt(tau) / 2

            d2 = -d - sig * np.sqrt(tau) / 2

            price = K * stats.norm.cdf(d1, 0.0, 1.0) - np.exp(dft * tau) * S * stats.norm.cdf(d2, 0.0, 1.0)

            return price
    else:
        sys.exit("option type error")

def greeks_theo(name,type,S,K,tau,sig,dft=0):
    if name=='delta':
        res = optionprice(type,S,K,tau,sig,dft)-optionprice(type,S+1,K,tau,sig,dft)
    elif name=='theta':
        res = optionprice(type,S,K,tau+1,sig,dft)-optionprice(type,S,K,tau,sig,dft)
    elif name=='gamma':
        res = optionprice(type,S+1,K,tau,sig,dft)+optionprice(type,S-1,K,tau,sig,dft)-2*optionprice(type,S,K,tau,sig,dft)
    else:
        sys.exit("option type error")
    return res * 100


def daysLeft(expiration,date):
    #  exipration: in Epoch ms
    # date: in dt.date or datetime
    secondsToExpiration = (expiration / 1000 - int(date.strftime('%s')))
    daysToExpiration = secondsToExpiration / (24*3600)
    return daysToExpiration

def optionpriceRow(option,stockprice,date):
    # stockprice: float
    # date: in dt.date or datetime
    
    secondsToExpiration = (option.expirationDate / 1000 - int(date.strftime('%s')))
    daysToExpiration = secondsToExpiration / (24*3600)
    return optionprice(option.putCall,stockprice,option.strikePrice,daysToExpiration,option.volatility)


def linear_price_range(latestPrice,percent=0.01,stepNum=20):
    low = latestPrice*((1-percent))
    high = latestPrice*((1+percent))
    step = (high-low) / stepNum
    priceRange = [ low + step*num for num in range(stepNum+1)]
    return priceRange

def option_stockprice_dates_DF(priceRange,dateRange,options):
    # https://stackoverflow.com/questions/42168103/how-to-expand-flatten-pandas-dataframe-efficiently
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html

    """
    priceRange: list
    dateRange: list in Epoch(s)
    options: dataframe
    """
    options['stockPrice'] = options.apply(lambda df:priceRange,axis=1)
    options['dates'] = options.apply(lambda df:dateRange,axis=1)
    newdf = options.explode('stockPrice').explode('dates')
    return newdf

In [5]:
n = 5

options = df.sample(n)

stockprice = 700

date = dt.date(2022,6,4)

In [6]:


def OptionPriceVis(options,stockprice,date):
    """
    options: dataframe of options (pandas dataframe)
    stockprice: prediction of the underlying stock price (number)
    date: prediction date (dt.date or dt.datetime)
    """

    options['ExpectedPrice'] = options.apply(lambda df: optionpriceRow(df,stockprice,date),axis=1)


    options['Num'] = options.apply(lambda df: 1,axis=1)


    expected = alt.Chart(options).mark_bar(opacity=0.5).encode(
        y='symbol:N',
        x='ExpectedPrice:Q',
        tooltip = ['description','ExpectedPrice','mark'],
    )

    putCall_color = alt.Color('putCall:N',scale=alt.Scale(domain=['PUT','CALL'],range=['red','green']))


    mark = alt.Chart(options).mark_tick(thickness=3).encode(
        y='symbol:N',
        x='mark:Q',
        color = putCall_color
    )

    return mark + expected

OptionPriceVis(options,stockprice,date)

alt.LayerChart(...)

In [7]:
# PnL diagram

options = df.sample(5)

centerPrice = 800

date = dt.date(2022,6,6)

def Option_PnL_Vis(options,date,centerPrice):
    """
    options: dataframe of options (pandas dataframe)
    date: date of PnL  (dt.date or dt.datetime)
    centerPrice: The visualization will draw the PnL diagram around centerPrice
    """



    def option_stockprice_DF(centerPrice,options,stepNum=500,percent=0.8):
        # https://stackoverflow.com/questions/42168103/how-to-expand-flatten-pandas-dataframe-efficiently
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html

        """
        Create a dataframe of options with varying stockPrice
        """
        low = centerPrice*((1-percent))
        high = centerPrice*((1+percent))
        step = (high-low) / stepNum
        priceRange = [ low + step*num for num in range(stepNum+1)]
        options['stockPrice'] = options.apply(lambda df:priceRange,axis=1)
        newdf = options.explode('stockPrice')
        return newdf


    optionss = option_stockprice_DF(centerPrice,options)

    optionss['ExpectedPrice'] = optionss.apply(lambda df: optionpriceRow(df,df.stockPrice,date),axis=1)

    optionss['Num'] = options.apply(lambda df: 1,axis=1) # number of options for the position

    optionss['Return'] = optionss.apply(lambda df: (df.ExpectedPrice - df.mark)*df.Num,axis=1)

    indivual = alt.Chart(optionss).encode(
        x='stockPrice:Q',
        y='Return:Q',
        color='symbol:N',
        tooltip = ['description','stockPrice','Return','Num']
    )

    # https://altair-viz.github.io/gallery/multiline_highlight.html

    highlight = alt.selection(type='single', on='mouseover',
                        fields=['symbol'], nearest=True)

    points = indivual.mark_circle().encode(
        opacity=alt.value(0)
    ).add_selection(
        highlight
    )

    lines = indivual.mark_line().encode(size=alt.condition(~highlight, alt.value(1), alt.value(3)))

    res = (points+lines).interactive()



    # res = alt.concat(res).properties(
    #     title=alt.TitleParams(
    #         ['black line is the overall PnL'],
    #         baseline='bottom',
    #         orient='bottom',
    #         anchor='end',
    #         fontWeight='normal',
    #         fontSize=10,

    #         )
    # )


    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
    # https://stackoverflow.com/questions/20490274/how-to-reset-index-in-a-pandas-dataframe
    totalReturnDF = optionss[['stockPrice','Return']].groupby(['stockPrice']).sum().reset_index()


    total = alt.Chart(totalReturnDF).mark_line(color='black').encode(
        x='stockPrice',
        y=alt.Y('Return:Q',title='Total Return'),
    )

    # https://altair-viz.github.io/gallery/multiline_tooltip.html

    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                            fields=['stockPrice'], empty='none')

    selectors = alt.Chart(totalReturnDF).mark_point().encode(
        x='stockPrice:Q',
        opacity=alt.value(0),
    ).add_selection(
        nearest
    )

    points = total.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    text = total.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, 'Return:Q', alt.value(' '))
    )

    rules = alt.Chart(totalReturnDF).mark_rule(color='gray').encode(
        x='stockPrice:Q'
    ).transform_filter(
        nearest
    )

    total = alt.layer(total, selectors, rules, points, text)


    res = res.properties(width=600) & total.properties(width=600)

    return res

Option_PnL_Vis(options,date,centerPrice)



alt.VConcatChart(...)

In [8]:
# Greeks Vis


def Greeks_Vis(options,stockprice,date):
    """
    options: dataframe of options (pandas dataframe)
    stockprice: prediction of the underlying stock price (number)
    date: prediction date (dt.date or dt.datetime)
    """

    options['Num'] = options.apply(lambda df: 1,axis=1)

    options['delta_theo'] = options.apply(lambda df: greeks_theo('delta',df.putCall,stockprice,df.strikePrice,daysLeft(df.expirationDate,date),df.volatility),axis=1)
    options['theta_theo'] = options.apply(lambda df: greeks_theo('theta',df.putCall,stockprice,df.strikePrice,daysLeft(df.expirationDate,date),df.volatility),axis=1)
    options['gamma_theo'] = options.apply(lambda df: greeks_theo('gamma',df.putCall,stockprice,df.strikePrice,daysLeft(df.expirationDate,date),df.volatility),axis=1)


    delta_vis = alt.Chart(options).mark_bar(opacity=0.5).encode(
        y='symbol:N',
        x='delta_theo:Q',
        tooltip = ['description','delta_theo'],
    )

    theta_vis = alt.Chart(options).mark_bar(opacity=0.5).encode(
        y='symbol:N',
        x='theta_theo:Q',
        tooltip = ['description','theta_theo'],
    )

    gamma_vis = alt.Chart(options).mark_bar(opacity=0.5).encode(
        y='symbol:N',
        x='gamma_theo:Q',
        tooltip = ['description','gamma_theo'],
    )

    return delta_vis & theta_vis & gamma_vis

Greeks_Vis(options,stockprice,date)

alt.VConcatChart(...)

In [14]:
# Greeks Table Vis

# https://altair-viz.github.io/gallery/binned_heatmap.html
days = 30

n = 2

options = df.sample(n)

# secondsToExpiration = (option.expirationDate / 1000 - int(date.strftime('%s')))
lastprice = 750

priceRange = linear_price_range(lastprice,stepNum=20,percent=0.2)

def Greek_Table_Vis(options,priceRange,days=30,greek='delta'):
    """
    Visualize future options greeks in {days} days
    options: dateframe
    days: positive integer
    priceRange: list of prices
    greek: greek name to visualize 
    """

    dateRange = [1000*int((dt.date.today()+dt.timedelta(days=i)).strftime('%s')) for i in range(days)]


    df_greeksTable = option_stockprice_dates_DF(priceRange,dateRange,options)


    def calculateGreeks(options):
        for greek in ['delta','theta','gamma']:
            options['{}_theo'.format(greek)] = options.apply(lambda df: greeks_theo(greek,df.putCall,stockprice,df.strikePrice,daysLeft(df.expirationDate,dt.datetime.fromtimestamp(df.dates/1000)),df.volatility),axis=1)

    calculateGreeks(df_greeksTable)

    GreeksDF = df_greeksTable[['stockPrice','delta_theo','theta_theo','gamma_theo','dates']].groupby(['stockPrice','dates']).sum().reset_index()


    # https://altair-viz.github.io/user_guide/times_and_dates.html 
    # https://altair-viz.github.io/user_guide/transform/timeunit.html#user-guide-timeunit-transform
    a = alt.Chart(GreeksDF).mark_rect().encode(
        alt.X('monthdate(dates):O'),
        alt.Y('stockPrice:O',scale=alt.Scale(zero=False)),
        alt.Color('{}_theo:Q'.format(greek), scale=alt.Scale(scheme='greenblue')),
        tooltip=['dates:T','stockPrice','delta_theo','theta_theo','gamma_theo','dates']
    ).properties(width=600)


    b = Greeks_Vis(options,stockprice,date)

    return a & b 
# alt.Chart(df_greeksTable).mark_point
# ().encode(
#     alt.X('dates:T',axis=alt.Axis(format='%m/%d',labelAngle=-45,title='date')),
#     alt.Y('stockPrice:N'))

Greek_Table_Vis(options,priceRange)

alt.VConcatChart(...)

In [10]:
# [dt.datetime.fromtimestamp((dt.date.today()+dt.timedelta(days=i)).timestamp() ) for i in range(days)]


In [11]:
# # Check dates form changing

# dt.datetime.fromtimestamp(dt.datetime.now().timestamp())

# dt.datetime.now().strftime('%s%Y')

# dateRange = [int((dt.datetime.now()+dt.timedelta(days=i)).strftime('%s')) for i in range(days)]

# [dt.datetime.fromtimestamp(_) for _ in dateRange]

[datetime.datetime(2022, 5, 29, 19, 19, 39),
 datetime.datetime(2022, 5, 30, 19, 19, 39),
 datetime.datetime(2022, 5, 31, 19, 19, 39),
 datetime.datetime(2022, 6, 1, 19, 19, 39),
 datetime.datetime(2022, 6, 2, 19, 19, 39),
 datetime.datetime(2022, 6, 3, 19, 19, 39),
 datetime.datetime(2022, 6, 4, 19, 19, 39),
 datetime.datetime(2022, 6, 5, 19, 19, 39),
 datetime.datetime(2022, 6, 6, 19, 19, 39),
 datetime.datetime(2022, 6, 7, 19, 19, 39),
 datetime.datetime(2022, 6, 8, 19, 19, 39),
 datetime.datetime(2022, 6, 9, 19, 19, 39),
 datetime.datetime(2022, 6, 10, 19, 19, 39),
 datetime.datetime(2022, 6, 11, 19, 19, 39),
 datetime.datetime(2022, 6, 12, 19, 19, 39),
 datetime.datetime(2022, 6, 13, 19, 19, 39),
 datetime.datetime(2022, 6, 14, 19, 19, 39),
 datetime.datetime(2022, 6, 15, 19, 19, 39),
 datetime.datetime(2022, 6, 16, 19, 19, 39),
 datetime.datetime(2022, 6, 17, 19, 19, 39),
 datetime.datetime(2022, 6, 18, 19, 19, 39),
 datetime.datetime(2022, 6, 19, 19, 19, 39),
 datetime.datetime(